In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
import pandas as pd

In [44]:
df = pd.read_csv("/content/drive/MyDrive/music recommendation model/spotify_millsongdata.csv")

In [45]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [46]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [47]:
df.shape

(57650, 4)

In [48]:
df.isnull().sum()

,0
artist,0
song,0
link,0
text,0


In [49]:
df = df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [50]:
df.shape

(5000, 3)

In [51]:
df.head(10)

,artist,song,text
0,Rush,Virtuality,Like a shipwrecked mariner adrift on an unknow...
1,Dan Fogelberg,Captured Angel,Found your heart \r\nAnd lost your lover \r\...
2,Chuck Berry,Beautiful Delilah,"Beautiful Delilah, sweet as apple pie \r\nAlw..."
3,One Direction,Stand Up,"From the moment I met you, everything changed ..."
4,Wanda Jackson,My Big Iron Skillet,You are doing wrong again it's plain for all t...
5,Oasis,Falling Down,The summer sun \r\nIt blows my mind \r\nIt's...
6,Robbie Williams,Dickhead,"This is ""T"" to the mother fucking ""Ofty"" \r\n..."
7,Cyndi Lauper,Across The Universe,Words are flowing out like endless rain into a...
8,Carly Simon,Alone Together,"Alone together, beyond the crowd \r\nAbove th..."
9,Blur,Dan Abnormal,Meanie Leanie come on down \r\nCome and enter...


In [52]:
df['text'][0]

"Like a shipwrecked mariner adrift on an unknown sea  \r\nClinging to the wreckage of the lost ship Fantasy  \r\nI'm a castaway, stranded in a desolate land  \r\nI can see the footprints in the virtual sand  \r\n  \r\nNet boy, net girl  \r\nSend your signal 'round the world  \r\nLet your fingers walk and talk  \r\nAnd set you free  \r\n  \r\nNet boy, net girl  \r\nSend your impulse 'round the world  \r\nPut your message in a modem  \r\nAnd throw it in the Cyber Sea  \r\n  \r\nAstronauts in the weightlessness of pixelated space  \r\nExchange graffiti with a disembodied race  \r\nI can save the universe in a grain of sand  \r\nI can hold the future in my virtual hand  \r\n  \r\nLet's dance tonight  \r\nTo a virtual song  \r\nPress this key  \r\nAnd you can play along  \r\n  \r\nLet's fly tonight  \r\nOn our virtual wings  \r\nPress this key  \r\nTo see amazing things  \r\n  \r\nLike a pair of vagabonds who wave between two passing trains  \r\nOr the glimpse of a woman's smile through a w

In [53]:
# Text Cleaning/ Preprocessing

In [54]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ',regex = True)

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer

In [ ]:
stemmer = PorterStemmer()

In [57]:
def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [58]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [60]:
tfid=TfidfVectorizer(analyzer='word', stop_words='english')

In [61]:
matrix=tfid.fit_transform(df['text'])

In [62]:
matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 276316 stored elements and shape (5000, 17573)>

In [63]:
similarity = cosine_similarity(matrix)

In [64]:
similarity[0]

array([1.        , 0.02961021, 0.00534931, ..., 0.04352872, 0.00156694,
       0.0349023 ])

In [66]:
df[df['song']== 'Stand Up'].index[0]

np.int64(3)

In [67]:
#Recommender Function

In [68]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [71]:
recommendation("Stand Up")

['Run Like A Thief',
 'All Along The Watchtower',
 'All Along The Watchtower (featuring Chrissie Hynde)',
 'Sacrifice',
 'Since I Lost You',
 'Crossroads',
 "Don't Let Him Steal Your Heart Away",
 'I Wish My Heart Could Talk',
 'Crazy Love',
 'I Know You By Heart',
 'Aftermath',
 'Piece Of My Heart',
 'Farther Away',
 "Don't Break My Heart",
 "I Don't Have The Heart",
 'On The Verge',
 'Hand On My Heart',
 'Hearts',
 'Left 2 3 4',
 "Livin' In A Heart"]

In [ ]:
import pickle

In [ ]:
pickle.dump(similarity, open("similarity.pkl", 'wb'))

In [ ]:
pickle.dump(df, open("df.pkl;", 'wb'))